In [ ]:
import sympy
import numpy as np
from math import factorial
from sympy.interactive import printing

printing.init_printing(use_unicode=True) 

In [ ]:
sympy.__version__

In [ ]:
from time import time

In [ ]:
wanted_order = 9

In [ ]:
t1 = time()

# Defining symbols
H0_AA, H0_BB, H1_AA, H1_BB, H2_AB, H2_BA, S_AB, S_BA = sympy.symbols('H0_AA, H0_BB, H1_AA, H1_BB, H2_AB, H2_BA, S_AB, S_BA', commutative=False)
x = sympy.Symbol('x', commutative=True)
S_AB_i = [sympy.symbols('S_AB_{}'.format(i), commutative=False) for i in range(0, wanted_order + 1)]
S_BA_i = [sympy.symbols('S_BA_{}'.format(i), commutative=False) for i in range(0, wanted_order + 1)]

S_AB_series = 0
S_BA_series = 0
for i in range(1, wanted_order + 1):
    S_AB_series += S_AB_i[i] * x ** (i)
    S_BA_series += S_BA_i[i] * x ** (i)

def commute_n_odd(H_AA, H_BB, S_AB, S_BA, n):
    assert n % 2 == 1
    res_AB = clean_terms(H_AA * S_AB - S_AB * H_BB, wanted_order)
    res_BA = clean_terms(H_BB * S_BA - S_BA * H_AA, wanted_order)
    for i in range((n - 1) // 2):
        res_AA = clean_terms(res_AB * S_BA - S_AB * res_BA, wanted_order)
        res_BB = clean_terms(res_BA * S_AB - S_BA * res_AB, wanted_order)
        res_AB = clean_terms(res_AA * S_AB - S_AB * res_BB, wanted_order)
        res_BA = clean_terms(res_BB * S_BA - S_BA * res_AA, wanted_order)
    return res_AB

def commute_n_even(H_AB, H_BA, S_AB, S_BA, n):
    assert n % 2 == 0
    res_AB = H_AB
    res_BA = H_BA
    for i in range(n // 2):
        res_AA = clean_terms(res_AB * S_BA - S_AB * res_BA, wanted_order)
        res_BB = clean_terms(res_BA * S_AB - S_BA * res_AB, wanted_order)
        res_AB = clean_terms(res_AA * S_AB - S_AB * res_BB, wanted_order)
        res_BA = clean_terms(res_BB * S_BA - S_BA * res_AA, wanted_order)
    return res_AB

def clean_terms(A, wanted_order):
    A = A.expand()
    return sum([A.coeff(x, n) * x**n for n in range(0, wanted_order+1)]).expand()

# Calculating offdiagonal 
Hn_AB = 0
for j in range(wanted_order//2 + 1):
    Hn_AB += commute_n_odd(H0_AA + H1_AA*x, H0_BB + H1_BB*x, S_AB_series, S_BA_series, 2*j+1) / sympy.factorial(2*j+1)
    Hn_AB += commute_n_even(H2_AB*x, H2_BA*x, S_AB_series, S_BA_series, 2*j)  / sympy.factorial(2*j)
    print('j = {} done'.format())
    
print('step 2')

Hn_AB = clean_terms(Hn_AB, wanted_order)

t2 = time()
print((t2-t1)/60)

In [ ]:
Y_i = []
for n in range(1, wanted_order + 1):
    Y = - (Hn_AB.coeff(x, n) - (H0_AA*S_AB_i[n] - S_AB_i[n]*H0_BB))
    # Y = - (collected[x**n])
    Y = Y.expand()
    display(Y)
    Y_i.append(Y)

# for computations

In [ ]:
def write_n(n):
    expr = - (Hn_AB.coeff(x, n) - (H0_AA*S_AB_i[n] - S_AB_i[n]*H0_BB))
    expr.expand()
    
    output = ""
    output += 'def Y%i(H0_AA, H0_BB, H1_AA, H1_BB, H2_AB, H2_BA, S_AB_i, S_BA_i):\n' % n
    if n > 1:
        output += '    '
        for i in range(1,n):
            output += 'S_AB_%i, ' % i
        output += '= S_AB_i[:%i]\n' % (n-1)
        output += '    '
        for i in range(1,n):
            output += 'S_BA_%i, ' % i
        output += '= S_BA_i[:%i]\n' % (n-1)
    output += '    return (' + str(expr) + ')\n'
    return output

In [ ]:
text_to_write = u""

for n in range(1, wanted_order+1):
    text_to_write += write_n(n)
    text_to_write += '\n\n'

text_to_write += 'Y_i = ('
for n in range(1, wanted_order+1):
    text_to_write += 'Y%i,' % n
text_to_write += ')\n'
    
import io

with io.open('perturbationS.py', 'w') as f:
    f.write(text_to_write)    